In [12]:
import pandas as pd

olid_test_labels_df = pd.read_csv('OLIDv1.0/labels-levela.csv', sep=',', names=['id', 'label'])
olid_test_tweets_df = pd.read_csv('OLIDv1.0/testset-levela.tsv', sep='\t')

In [13]:
olid_test_labels_df['label'] = olid_test_labels_df['label'].map({'OFF': 1, 'NOT': 0})

In [14]:
# olid_test_labels_df.head()
merged_df = pd.merge(olid_test_tweets_df, olid_test_labels_df, on='id')


In [19]:
merged_df = merged_df.drop(columns=['id'])

In [20]:

def remap_labels(df, col_name):
    df[col_name] = df[col_name].map({'OFF': 1, 'NOT': 0})
    return df
def remap_HSO_labels(df):
    df['class'] = df['class'].map({2: 0, 1: 1, 0: 1}) 
    return df

def remap_tweets(df, col_name):
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('URL', '') #Remove URLs
    pat = r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])' # remove URLS
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, ' ', regex=True)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&lt','') #Remove &lt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&gt','') #Remove &gt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\n',' ') #Remove \n
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\d+','', regex=True) #Remove numbers
    pat = r'[^\w\s]' # remove the emojis and other special characters
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, '', regex=True)
    #remove white space before and after the tweet
    df.loc[:, col_name] = df.loc[:, col_name].str.strip()
    df.loc[:, col_name] = df.loc[:, col_name].str.lower() #Lowercase
    
    # not needed outside of english
    contractions_dict = {
        r"\bive\b": "i have",
        r"\byoure\b": "you are",
        r"\bitd\b": "it would",
        r"\bcant\b": "cannot",
        r"\bdont\b": "do not",
        r"\bwont\b": "will not",
        r"\btho\b": "though",
        r"\bim\b": "i am",
        r"\bhes\b": "he is",
        r"\bshes\b": "she is",
        r"\bits\b": "it is",
        r"\bwere\b": "we are",
        r"\btheyre\b": "they are",
        r"\bshouldnt\b": "should not",
        r"\bwouldnt\b": "would not",
        r"\bcouldnt\b": "could not",
        r"\bisnt\b": "is not",
        r"\barent\b": "are not",
        r"\bhavent\b": "have not",
        r"\bhasnt\b": "has not",
        r"\bhadnt\b": "had not",
        r"\bwasnt\b": "was not",
        r"\bwerent\b": "were not"
    }
    
    # df[col_name] = df[col_name].replace(contractions_dict, regex=True) # Replace some popular contractions
    
    
    return df

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_2813/73532889.py:17: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, col_name] = df.loc[:, col_name].str.replace('\d+','', regex=True) #Remove numbers


In [21]:
OLID_test = remap_tweets(merged_df, 'tweet')


In [22]:
OLID_test.head()

,tweet,label
0,whoisq wherestheserver dumpnike declasfisa dem...,1
1,constitutionday is revered by conservatives ha...,0
2,foxnews nra maga potus trump ndamendment rnc u...,0
3,watching boomer getting the news that she is s...,0
4,nopasaran unity demo to oppose the farright in...,1


In [23]:
OLID_test.to_csv('datasets/cleaned_OLID_test.tsv', sep='\t')